In [2]:
import numpy as np
import pandas as pd
import mglearn
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
from matplotlib import font_manager, rc
import seaborn as sns

D:\anaconda\lib\site-packages\sklearn\externals\six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
D:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
x = pd.read_csv("variable_total.csv")

In [4]:
x = x.drop(["Unnamed: 0"],axis=1)

In [5]:
x.shape

(1997, 79)

In [6]:
y_onehot = pd.read_csv("train_profile_onehot.csv")

In [7]:
y_onehot

,CUS_ID,F20-,F30,F40+,M20-,M30,M40+
0,1,0,0,0,0,0,1
1,2,1,0,0,0,0,0
2,4,0,0,0,0,1,0
3,5,0,0,0,0,0,1
4,8,0,0,0,0,1,0
...,...,...,...,...,...,...,...
1992,2494,0,0,0,1,0,0
1993,2495,0,1,0,0,0,0
1994,2496,0,0,0,0,0,1
1995,2498,0,0,1,0,0,0


In [8]:
y = pd.read_csv("train_profile_group.csv")

In [9]:
y

,CUS_ID,GROUP
0,1,5
1,2,0
2,4,4
3,5,5
4,8,4
...,...,...
1992,2494,3
1993,2495,1
1994,2496,5
1995,2498,2


---
train test 나누기 <br>
minmaxscaled

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [22]:
y=y.iloc[:,1]

In [23]:
y_list=y.values.tolist()

In [24]:
y_list=np.array(y_list)

In [25]:
y_list

array([5, 0, 4, ..., 5, 2, 3])

In [47]:
# onehot 없이
x_train,x_test,y_train,y_test = train_test_split(x,y_list,random_state=0)

In [48]:
scaler = MinMaxScaler()
scaler.fit(x_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [49]:
x_train_scaled =scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [50]:
y_train

array([2, 2, 5, ..., 1, 5, 2])

-----
원핫 한거 split

In [42]:
y_onehot=y_onehot.iloc[:,1:7]

In [43]:
y_list_one = y_onehot.values.tolist()

In [44]:
y_list_one = np.array(y_list_one)

In [51]:
x_train_one,x_test_one,y_train_one,y_test_one = train_test_split(x,y_list_one,random_state=0)

In [52]:
y_train_one

array([[0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0]])

In [56]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

------
knn

In [57]:
from sklearn.neighbors import KNeighborsClassifier

In [59]:
pipe = make_pipeline(MinMaxScaler(),KNeighborsClassifier())

In [60]:
pipe.steps

[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
 ('kneighborsclassifier',
  KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                       metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                       weights='uniform'))]

In [77]:
param_grid = {'kneighborsclassifier__n_neighbors':[1,3,5,7,9]}

In [78]:
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x_train,y_train)
grid.best_score_

0.28121739130434786

In [79]:
grid.score(x_test,y_test)

0.304

In [86]:
# 원핫 한거
grid.fit(x_train_one,y_train_one)
grid.best_score_

0.2398015607580825

In [88]:
grid.score(x_test_one,y_test_one)

0.27

In [89]:
grid.best_params_

{'kneighborsclassifier__n_neighbors': 1}

In [81]:
# scaled 없이
clf = KNeighborsClassifier(n_neighbors=9)

In [82]:
clf.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')

In [83]:
clf.score(x_test,y_test)

0.268

In [90]:
# scaled 없이 원핫 o
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(x_train_one,y_train_one)
clf.score(x_test_one,y_test_one)

0.204

In [91]:
from sklearn.neighbors import KNeighborsRegressor

In [92]:
pipe = make_pipeline(MinMaxScaler(),KNeighborsRegressor())
param_grid = {'kneighborsregressor__n_neighbors':[1,3,5,7,9]}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x_train,y_train)
print(grid.best_score_)
print(grid.score(x_test,y_test))
print(grid.best_params_)

0.01390965079719333
-0.023713195341958926
{'kneighborsregressor__n_neighbors': 9}


In [93]:
# scaled 없이
clf = KNeighborsRegressor(n_neighbors=9)
clf.fit(x_train,y_train)
clf.score(x_test,y_test)

-0.07376076149821209

In [94]:
# onehot o
grid.fit(x_train_one,y_train_one)
print(grid.best_score_)
print(grid.score(x_test_one,y_test_one))
print(grid.best_params_)

D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_av

-0.05481960212304911
-0.03597564098566394
{'kneighborsregressor__n_neighbors': 9}


D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [95]:
# scaled 없이 원핫 o
clf = KNeighborsRegressor(n_neighbors=9)
clf.fit(x_train_one,y_train_one)
clf.score(x_test_one,y_test_one)

D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


-0.07924042811978084

----
선형

In [96]:
from sklearn.linear_model import LinearRegression

In [98]:
pipe = make_pipeline(MinMaxScaler(),LinearRegression())
pipe.fit(x_train,y_train)
print(pipe.score(x_test,y_test))

-0.06324316904234117


In [99]:
aa = LinearRegression()
aa.fit(x_train,y_train)
aa.score(x_test,y_test)

0.10466902725309124

In [101]:
# onehot o
pipe.fit(x_train_one,y_train_one)
print(pipe.score(x_test_one,y_test_one))

-0.10112732983957881


D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [102]:
# scaled 없이 원핫 o
aa = LinearRegression()
aa.fit(x_train_one,y_train_one)
aa.score(x_test_one,y_test_one)

D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


-0.02832493340449153

---
리지 회귀

In [103]:
from sklearn.linear_model import Ridge

In [104]:
pipe = make_pipeline(MinMaxScaler(),Ridge())
param_grid = {'ridge__alpha':[0.01,0.1,1,10,100]}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x_train,y_train)
print(grid.best_score_)
print(grid.score(x_test,y_test))
print(grid.best_params_)

0.10999189036817122
0.12132097462725443
{'ridge__alpha': 10}


In [105]:
aa = Ridge(alpha=10)
aa.fit(x_train,y_train)
aa.score(x_test,y_test)

0.10466941416425435

In [106]:
grid.fit(x_train_one,y_train_one)
print(grid.best_score_)
print(grid.score(x_test_one,y_test_one))
print(grid.best_params_)

0.03823063772840862
0.05753043073026931
{'ridge__alpha': 10}


D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_av

In [107]:
aa.fit(x_train_one,y_train_one)
aa.score(x_test_one,y_test_one)

D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


-0.028324279031704827

----
라소 회귀

In [108]:
from sklearn.linear_model import Lasso

In [110]:
pipe = make_pipeline(MinMaxScaler(),Lasso(max_iter=100000))
param_grid = {'lasso__alpha':[0.001,0.01,0.1,1,10,100]}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x_train,y_train)
print(grid.best_score_)
print(grid.score(x_test,y_test))
print(grid.best_params_)

0.10608872863669602
0.10342706668056212
{'lasso__alpha': 0.01}


In [115]:
aa = Lasso(alpha=0.01,max_iter=100000)
aa.fit(x_train,y_train)
aa.score(x_test,y_test)

0.10516061515387264

In [113]:
grid.fit(x_train_one,y_train_one)
print(grid.best_score_)
print(grid.score(x_test_one,y_test_one))
print(grid.best_params_)

D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_av

0.03394478070308132
0.05708075051356726
{'lasso__alpha': 0.001}


D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_av

In [117]:
aa = Lasso(alpha=0.001,max_iter=100000)

In [118]:
aa.fit(x_train_one,y_train_one)
aa.score(x_test_one,y_test_one)

D:\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 124.14911968055542, tolerance: 0.031108617234468946
  positive)
D:\anaconda\lib\site-packages\sklearn\base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


-0.027982177240337112

----
로지스틱 회귀

In [119]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [121]:
pipe = make_pipeline(MinMaxScaler(),LogisticRegression(max_iter=100000))
param_grid = {'logisticregression__C':[0.001,0.01,0.1,1,10,100]}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x_train,y_train)
print(grid.best_score_)
print(grid.score(x_test,y_test))
print(grid.best_params_)

0.3660735785953177
0.382
{'logisticregression__C': 1}


In [126]:
aa = LogisticRegression(C=1,max_iter=1000000)
aa.fit(x_train,y_train)
aa.score(x_test,y_test)

D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.368

----
linearSVC

In [128]:
pipe = make_pipeline(MinMaxScaler(),LinearSVC(max_iter=100000))
param_grid = {'linearsvc__C':[0.001,0.01,0.1,1,10,100]}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x_train,y_train)
print(grid.best_score_)
print(grid.score(x_test,y_test))
print(grid.best_params_)

0.3573667781493869
0.374
{'linearsvc__C': 10}


In [138]:
aa = LinearSVC(C=10,max_iter=100000)
aa.fit(x_train,y_train)
aa.score(x_test,y_test)

D:\anaconda\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.2

---
결정 트리

In [132]:
from sklearn.tree import DecisionTreeClassifier

In [133]:
pipe = make_pipeline(MinMaxScaler(),DecisionTreeClassifier())
param_grid = {'decisiontreeclassifier__max_depth':[4,6,8,10,12,14,16]}
grid= GridSearchCV(pipe,param_grid=param_grid,cv=5)
grid.fit(x_train,y_train)
print(grid.best_score_)
print(grid.score(x_test,y_test))
print(grid.best_params_)

0.31530434782608696
0.336
{'decisiontreeclassifier__max_depth': 4}


In [140]:
aa = DecisionTreeClassifier(max_depth=4)
aa.fit(x_train,y_train)
aa.score(x_test,y_test)

0.336

In [141]:
grid.fit(x_train_one,y_train_one)
print(grid.best_score_)
print(grid.score(x_test_one,y_test_one))
print(grid.best_params_)

0.26855741360089186
0.254
{'decisiontreeclassifier__max_depth': 14}


In [137]:
aa = DecisionTreeClassifier(max_depth=14)
aa.fit(x_train_one,y_train_one)
aa.score(x_test_one,y_test_one)

0.252

In [142]:
aa.feature_importances_

array([0.        , 0.02469185, 0.        , 0.05536018, 0.        ,
       0.08794076, 0.        , 0.        , 0.        , 0.06184029,
       0.        , 0.20411489, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.04244775, 0.        , 0.08865545,
       0.04920955, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.06315562,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.10309628, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.03420

In [154]:
a=x.columns.tolist()

In [160]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'Malgun Gothic'

In [1]:
def plot_feature_importances_cancer(model):
    n_features = x.shape[1]
    plt.barh(np.arange(n_features), model.feature_importances_,align='center')
    plt.yticks(np.arange(n_features), a)
    plt.xlabel("특성중요도")
    plt.ylabel("특성")
    plt.ylim(-1,n_features)
    plt.figure(figsize=(120, 50))
plot_feature_importances_cancer(aa)

NameError: name 'aa' is not defined